In [1]:
import pandas as pd
import csv
from anytree import Node, Walker
import re
import ast
import swifter
import json

In [2]:
eu_tags = pd.read_csv("filtered_eu_codes.csv")
eu_tags["name_list"] = eu_tags["name_list"].apply(lambda x: ast.literal_eval(x))

In [3]:
eu_false_positives = re.compile("|".join(["[furniture|nail] polish", # ethnicity FPs
                                         "irish [whiskey|cream]",
                                         "speak[er|ing]",
                                         "language",
                                         "hispanic",
                                         "tour de france",
                                         "greek joghurt",

                                         # Country code FPs
                                         "accuen", # does market research and very little location-specifics. Thus many FPs
                                         "xaxisus", # us source
                                         "xaxisca", # canadian source
                                         "xaxisapc", # whatever the fuck it is, it's not useful
                                         "tailtarget", # mostly latAm focussed -> numerous es/pt FPs
                                         "\A\d{20}",
                                         ]))

eu_tags_filtered = eu_tags[eu_tags.swifter.apply(lambda x: not bool(re.search(eu_false_positives, x["name"])), axis = 1)]

Pandas Apply:   0%|          | 0/20911 [00:00<?, ?it/s]

In [4]:
tree = {}

for item, row in eu_tags_filtered.iterrows():
    segment_list = row["name_list"]
    level = tree
    for item in segment_list:
        if not item in level:
            level[item] = {}
        level = level[item]

In [5]:
def squeeze_tree(tree, thresh):
    items = list(tree.items())
    if len(items) == 1:
        k,v = items[0]

        if len(v) == 0:
            return k
        
        ret = squeeze_tree(v, thresh)
        if type(ret) == str:
            return " - ".join([k, ret])
        else:
            return {k: ret}

    temp = {}
    for k, v in items:
        temp[k] = squeeze_tree(v, thresh)
    return temp

In [6]:
tree_squeezed = squeeze_tree(tree, 1)
with open("hierarchy.json", "w") as f:
    json.dump(tree_squeezed, f)

In [11]:
def dict_n_leaves(in_dict):
    if type(in_dict) == str or len(in_dict) == 1:
        return 1
    return sum([dict_n_leaves(v) for k,v in in_dict.items()])


tree_shallow = {}
tree_deep = {}

def filter_by_depth(tree, thresh):
    for k,v in tree.items():
        if dict_n_leaves(v) < thresh:
            tree_shallow[k] = v
        else:
            tree_deep[k] = v

filter_by_depth(tree_squeezed, 4)

In [12]:
with open("hierarchy_deep.json", "w") as f:
    json.dump(tree_deep, f)